In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install qwen-vl-utils
!pip install qwen_agent
!pip install openai

In [2]:
# Get Noto font
# !apt-get install fonts-noto-cjk

import os
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
from bs4 import BeautifulSoup, Tag
from pathlib import Path
import re

# Function to draw bounding boxes and text on images based on HTML content
def draw_bbox(image_path, resized_width, resized_height, full_predict):
    if image_path.startswith("http"):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(image_path)
    original_width = image.width
    original_height = image.height

    # Parse the provided HTML content
    soup = BeautifulSoup(full_predict, 'html.parser')
    # Extract all elements that have a 'data-bbox' attribute
    elements_with_bbox = soup.find_all(attrs={'data-bbox': True})

    filtered_elements = []
    for el in elements_with_bbox:
        if el.name == 'ol':
            continue  # Skip <ol> tags
        elif el.name == 'li' and el.parent.name == 'ol':
            filtered_elements.append(el)  # Include <li> tags within <ol>
        else:
            filtered_elements.append(el)  # Include all other elements

    font = ImageFont.truetype("NotoSansCJK-Regular.ttc", 20)
    draw = ImageDraw.Draw(image)

    # Draw bounding boxes and text for each element
    for element in filtered_elements:
        bbox_str = element['data-bbox']
        text = element.get_text(strip=True)
        x1, y1, x2, y2 = map(int, bbox_str.split())

        # Calculate scaling factors
        scale_x = resized_width / original_width
        scale_y = resized_height / original_height

        # Scale coordinates accordingly
        x1_resized = int(x1 / scale_x)
        y1_resized = int(y1 / scale_y)
        x2_resized = int(x2 / scale_x)
        y2_resized = int(y2 / scale_y)

        if x1_resized > x2_resized:
            x1_resized, x2_resized = x2_resized, x1_resized
        if y1_resized > y2_resized:
            y1_resized, y2_resized = y2_resized, y1_resized

        # Draw bounding box
        draw.rectangle([x1_resized, y1_resized, x2_resized, y2_resized], outline='red', width=2)
        # Draw associated text
        draw.text((x1_resized, y2_resized), text, fill='black', font=font)

    # Display the image
    image.show()

# Function to clean and format HTML content
def clean_and_format_html(full_predict):
    soup = BeautifulSoup(full_predict, 'html.parser')

    # Regular expression pattern to match 'color' styles in style attributes
    color_pattern = re.compile(r'\bcolor:[^;]+;?')

    # Find all tags with style attributes and remove 'color' styles
    for tag in soup.find_all(style=True):
        original_style = tag.get('style', '')
        new_style = color_pattern.sub('', original_style)
        if not new_style.strip():
            del tag['style']
        else:
            new_style = new_style.rstrip(';')
            tag['style'] = new_style

    # Remove 'data-bbox' and 'data-polygon' attributes from all tags
    for attr in ["data-bbox", "data-polygon"]:
        for tag in soup.find_all(attrs={attr: True}):
            del tag[attr]

    classes_to_update = ['formula.machine_printed', 'formula.handwritten']
    # Update specific class names in div tags
    for tag in soup.find_all(class_=True):
        if isinstance(tag, Tag) and 'class' in tag.attrs:
            new_classes = [cls if cls not in classes_to_update else 'formula' for cls in tag.get('class', [])]
            tag['class'] = list(dict.fromkeys(new_classes))  # Deduplicate and update class names

    # Clear contents of divs with specific class names and rename their classes
    for div in soup.find_all('div', class_='image caption'):
        div.clear()
        div['class'] = ['image']

    classes_to_clean = ['music sheet', 'chemical formula', 'chart']
    # Clear contents and remove 'format' attributes of tags with specific class names
    for class_name in classes_to_clean:
        for tag in soup.find_all(class_=class_name):
            if isinstance(tag, Tag):
                tag.clear()
                if 'format' in tag.attrs:
                    del tag['format']

    # Manually build the output string
    output = []
    for child in soup.body.children:
        if isinstance(child, Tag):
            output.append(str(child))
            output.append('\n')  # Add newline after each top-level element
        elif isinstance(child, str) and not child.strip():
            continue  # Ignore whitespace text nodes
    complete_html = f"""```html\n<html><body>\n{" ".join(output)}</body></html>\n```"""
    return complete_html

In [ ]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
model_path = "Qwen/Qwen2.5-VL-3B-Instruct"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_path, torch_dtype=torch.bfloat16,device_map="auto")
processor = AutoProcessor.from_pretrained(model_path)

In [10]:
def inference(img_url, prompt, system_prompt="You are a helpful assistant", max_new_tokens=32000):
  image = Image.open(img_url)
  messages = [
    {
      "role": "system",
      "content": system_prompt
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt
        },
        {
          "image": img_url
        }
      ]
    }
  ]
  device = "cuda" if torch.cuda.is_available() else "cpu"
  text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  print("input:\n",text)
  inputs = processor(text=[text], images=[image], padding=True, return_tensors="pt").to(device)

  output_ids = model.generate(**inputs, max_new_tokens=1024)
  generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
  output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
  #print("output:\n",output_text[0])

  input_height = inputs['image_grid_thw'][0][1]*14
  input_width = inputs['image_grid_thw'][0][2]*14

  return output_text[0], input_height, input_width

In [ ]:
from openai import OpenAI
import os
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# @title inference function with API
def inference_with_api(image_path, prompt, sys_prompt="You are a helpful assistant.", model_id="qwen2.5-vl-3b-instruct", min_pixels=512*28*28, max_pixels=2048*28*28):
    base64_image = encode_image(image_path)
    client = OpenAI(
        #If the environment variable is not configured, please replace the following line with the Dashscope API Key: api_key="sk-xxx".
        api_key=os.getenv('DASHSCOPE_API_KEY'),
        base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
    )


    messages=[
        {
            "role": "system",
            "content": [{"type":"text","text": sys_prompt}]},
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "min_pixels": min_pixels,
                    "max_pixels": max_pixels,
                    # Pass in BASE64 image data. Note that the image format (i.e., image/{format}) must match the Content Type in the list of supported images. "f" is the method for string formatting.
                    # PNG image:  f"data:image/png;base64,{base64_image}"
                    # JPEG image: f"data:image/jpeg;base64,{base64_image}"
                    # WEBP image: f"data:image/webp;base64,{base64_image}"
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]
    completion = client.chat.completions.create(
        model = model_id,
        messages = messages,

    )
    return completion.choices[0].message.content

In [ ]:
# Imports PIL module
import urllib.request
from PIL import Image
import os
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/Engine_TestData')
#image = Image.open('N001.jpg')
#plt.imshow(image)

#img_url = "assets/document_parsing/docparsing_example6.png"
img_url = 'T001.jpg' #'/content/drive/My Drive/Colab Notebooks/Dataset/img/E009.jpg'
image = Image.open(img_url)
plt.imshow(image)
system_prompt="You are an AI specialized in recognizing and extracting text from images. Your mission is to analyze the image document and generate the result in QwenVL Document Parser HTML format using specified tags while maintaining user privacy and data integrity."
prompt =  "QwenVL Document Parser" #"QwenVL HTML"

## Use a local HuggingFace model to inference.
output, input_height, input_width = inference(img_url, prompt)


# Visualization
print(input_height, input_width)
print(output)


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO
from bs4 import BeautifulSoup, Tag
from pathlib import Path
import re

#draw_bbox(img_url, input_width, input_height, output)
image_path = img_url
full_predict = output
image = Image.open(image_path)
original_width = image.width
original_height = image.height
resized_width = input_width
resized_height = input_height
print(original_width, original_height)
print(resized_width, resized_height)

# Parse the provided HTML content
soup = BeautifulSoup(full_predict, 'html.parser')
# Extract all elements that have a 'data-bbox' attribute
elements_with_bbox = soup.find_all(attrs={'data-bbox': True})

filtered_elements = []
for el in elements_with_bbox:
    if el.name == 'ol':
        continue  # Skip <ol> tags
    elif el.name == 'li' and el.parent.name == 'ol':
        filtered_elements.append(el)  # Include <li> tags within <ol>
    else:
        filtered_elements.append(el)  # Include all other elements

#font = ImageFont.truetype('Roboto-Bold.ttf', size=10)
draw = ImageDraw.Draw(image)

# Draw bounding boxes and text for each element
for element in filtered_elements:
    bbox_str = element['data-bbox']
    text = element.get_text(strip=True)
    x1, y1, x2, y2 = map(int, bbox_str.split())

    # Calculate scaling factors
    scale_x = resized_width / original_width
    scale_y = resized_height / original_height

    # Scale coordinates accordingly
    x1_resized = int(x1 / scale_x)
    y1_resized = int(y1 / scale_y)
    x2_resized = int(x2 / scale_x)
    y2_resized = int(y2 / scale_y)

    if x1_resized > x2_resized:
        x1_resized, x2_resized = x2_resized, x1_resized
    if y1_resized > y2_resized:
        y1_resized, y2_resized = y2_resized, y1_resized

    # Draw bounding box
    draw.rectangle([x1_resized, y1_resized, x2_resized, y2_resized], outline='red', width=2)
    # Draw associated text
    draw.text((x1_resized, y2_resized), text, fill='black')

# Display the image
#image.show()
plt.imshow(image)

#ordinary_html = clean_and_format_html(output)
#soup = BeautifulSoup(full_predict, 'html.parser')

# Regular expression pattern to match 'color' styles in style attributes
color_pattern = re.compile(r'\bcolor:[^;]+;?')

# Find all tags with style attributes and remove 'color' styles
for tag in soup.find_all(style=True):
    original_style = tag.get('style', '')
    new_style = color_pattern.sub('', original_style)
    if not new_style.strip():
        del tag['style']
    else:
        new_style = new_style.rstrip(';')
        tag['style'] = new_style

# Remove 'data-bbox' and 'data-polygon' attributes from all tags
for attr in ["data-bbox", "data-polygon"]:
    for tag in soup.find_all(attrs={attr: True}):
        del tag[attr]

classes_to_update = ['formula.machine_printed', 'formula.handwritten']
# Update specific class names in div tags
for tag in soup.find_all(class_=True):
    if isinstance(tag, Tag) and 'class' in tag.attrs:
        new_classes = [cls if cls not in classes_to_update else 'formula' for cls in tag.get('class', [])]
        tag['class'] = list(dict.fromkeys(new_classes))  # Deduplicate and update class names

# Clear contents of divs with specific class names and rename their classes
for div in soup.find_all('div', class_='image caption'):
    div.clear()
    div['class'] = ['image']

classes_to_clean = ['music sheet', 'chemical formula', 'chart']
# Clear contents and remove 'format' attributes of tags with specific class names
for class_name in classes_to_clean:
    for tag in soup.find_all(class_=class_name):
        if isinstance(tag, Tag):
            tag.clear()
            if 'format' in tag.attrs:
                del tag['format']

# Manually build the output string
output1 = []
'''
for child in soup.body.children:
    if isinstance(child, Tag):
        output1.append(str(child))
        output1.append('\n')  # Add newline after each top-level element
    elif isinstance(child, str) and not child.strip():
        continue  # Ignore whitespace text nodes'''
complete_html = f"""```html\n<html><body>\n{" ".join(soup)}</body></html>\n```"""


print(complete_html)
